In [ ]:

import pandas as pd
import numpy as np

from cleaning_helpers import clean_column, same_name_merge

In [ ]:
print("""
     ## Warning, data files used in this code not included in repository.
     ## Company data files too large to push to github.
    """)

In [ ]:
# # Import all company lists
# extra_company_df = pd.read_csv('src/data/company_data/ExtraCompanies.csv', index_col=False)
# va_company_data = pd.read_csv('src/data/company_data/company_data_virginia.csv', on_bad_lines='skip', index_col=False)
# wburg_company_df = pd.read_csv('src/data/company_data/company_data_williamsburg.csv', index_col=False)
# wburg_company_df1 = pd.read_csv('src/data/company_data/company_data_williamsburg_POA.csv', index_col=False)
# # norfolk_company_df = pd.read_csv('src/data/company_data/company_data_norfolk.csv', index_col=False)
# #richmond_company_df = pd.read_csv('src/data/company_data/company_data_richmond_POA.csv', index_col=False)
# #herndon_company_df = pd.read_csv('src/data/company_data/company_data_herndon.csv', index_col=False)

# # Concatenate and Clean data
# company_df = pd.concat([extra_company_df,
#                         va_company_data, 
#                         wburg_company_df, 
#                         wburg_company_df1
#                         #norfolk_company_df, 
#                         #richmond_company_df, 
#                         #herndon_company_df, 
#                        ])
# company_df.columns = ['CID', 'company_name', 'name_type', 'entity_type', 'series_lcc', 'company_address', 'RA_name', 'status']
# company_df = company_df.drop(columns = ['name_type', 'entity_type', 'status', 'series_lcc'])
# company_df = company_df.drop_duplicates(subset = ['CID'], keep='first').reset_index(drop=True)
# # Save as CSV
# company_df.to_csv('src/data/data_cleaning/essential_company_data.csv', index=False)

In [ ]:
# Read rental_df
rental_df = pd.read_csv('src/data/data_cleaning/compiled_df.csv', index_col=False)

# Save Orginal Owner Names and Addresses for display
rental_df['owner_name_display'] = rental_df['owner_name']
rental_df['address_display'] = rental_df['address']

# # Clean Owner Name and Address
rental_df['owner_name'] = clean_column(rental_df, 'owner_name')
rental_df['address'] = clean_column(rental_df, 'address')

In [4]:
# Get all properties which might be owned by a corporation
companies = pd.DataFrame(
    rental_df[(rental_df['owner_name'].str.contains('lc')) | 
    (rental_df['owner_name'].str.contains('inc')) | 
    (rental_df['owner_name'].str.contains('plc')) |
    (rental_df['owner_name'].str.contains('corp'))|
    (rental_df['owner_name'].str.contains('company'))]['owner_name'].value_counts()).reset_index()

In [ ]:
# Read cleaned all company csv
company_df = pd.read_csv('src/data/data_cleaning/essential_company_data.csv', index_col=False)

In [6]:
# Clean company columns further
company_df['company_name_display'] = company_df['company_name']
company_df['company_name'] = clean_column(company_df, 'company_name')

company_df['ind_name_display'] = company_df['RA_name']
company_df['RA_name'] = clean_column(company_df, 'RA_name')

In [7]:
# Left Merge on 
merged = companies.merge(
    company_df[['company_name','CID','RA_name', 'ind_name_display', 'company_address']],
    left_on='owner_name',
    right_on='company_name',
    how='left'
)

# Then drop the extra company_name column and rename RA_name to ind_name
companies_merged = merged.drop(columns=['company_name']).rename(columns={'RA_name':'ind_name'})

In [8]:
# Outer Merge individual owner data from company data back into original dataframe
all_merged = pd.merge(rental_df, companies_merged, on = 'owner_name', how='outer')

# Replace NAs with blank strings
all_merged.loc[all_merged['ind_name'].isna(), 'ind_name'] = ''
all_merged.loc[all_merged['ind_name'].isna(), 'ind_name_display'] = ''

# Where blank string exists in ind_name, make owner_name ind_name
all_merged.loc[all_merged['ind_name']=='', 'ind_name'] = all_merged.loc[all_merged['ind_name']=='', 'owner_name']
all_merged.loc[all_merged['ind_name']=='', 'ind_name_display'] = all_merged.loc[all_merged['ind_name_display']=='', 'owner_name_display']


# Abreviate Living Trust to lt so Levenshtein ratio is less annoying
all_merged['ind_name'] = all_merged['ind_name'].str.replace('livingtrust', '_lt')
all_merged['ind_name'] = all_merged['ind_name'].str.replace('familytrust', '_ft')
all_merged['ind_name'] = all_merged['ind_name'].str.replace('revocabletrust', '_rt')
all_merged['ind_name'] = all_merged['ind_name'].str.replace('limitedliabilitycompany', '_llc')

In [9]:
# Set similar names to same name if they should be
clean_merged = same_name_merge(all_merged, 'ind_name', 'ind_name_display', 0.7)

jamesevaiden jamesandre


 0


zhaoqingzhang mingzhang


 0


joshuamdavid lamdavid


...

wrightbarry wrightmarke


 1


wrightmarkedward wrightmarkepamelah


 1


wrightmarkedward set to wrightmarkepamelah


In [14]:
# # See who owns the most units 
# clean_merged.groupby(['ind_name'], as_index=False)['rental_units'].sum().sort_values('rental_units', ascending=False).head(40)

In [ ]:
# Save to csv 
clean_merged.to_csv('src/data/data_cleaning/clean_merged.csv', index = False)

In [ ]:
clean_merged = pd.read_csv('src/data/data_cleaning/clean_merged.csv', index_col=False)

In [17]:
clean_merged[clean_merged['ind_name'].str.contains('pons')]

,PID,owner_name,address,acres,total_value,usage,half_baths,full_baths,bedrooms,rental_units,image_path,owner_name_display,address_display,count,CID,ind_name,ind_name_display,company_address
403,1062,goldenpondinc,101wytheave,0.18,373200,Single-Family (Rental),0.0,3.0,4.0,1.0,data/property_data/image_data/1062.png,"Golden Pond, Inc.",101 WYTHE AVE,1.0,7134893,douglasgpons,DOUGLAS G PONS,"614 College Ter, Williamsburg, VA, 23185 - 353..."
404,1062,goldenpondinc,101wytheave,0.18,373200,Single-Family (Rental),0.0,3.0,4.0,1.0,data/property_data/image_data/1062.png,"Golden Pond, Inc.",101 WYTHE AVE,1.0,07134893,douglasgpons,DOUGLAS G PONS,"614 College Ter, Williamsburg, VA, 23185 - 353..."


In [24]:
tOwner

,company_name,company_name_display,ind_name,ind_name_display,company_address
0,prometheusinvestmentsinc,PROMETHEUS INVESTMENTS INC,demetriosflorakis,DEMETRIOS FLORAKIS,"1220 RICHMOND RD, WILLIAMSBURG, VA, 23185 - 00..."
1,cs1031sterlingmanorapartmentsdst,"CS1031 Sterling Manor Apartments, DST",cs1031sterlingmanorapartmentsdst,"CS1031 Sterling Manor Apartments, DST",
2,dandylandllc,Dandy Land LLC,bennettlstein,Bennett L Stein,"632 Hampton Hwy, Yorktown, VA, 23693 - 4100, USA"
3,loneginc,LONE-G INC,gregoryhgranger,GREGORY H GRANGER,"1005 RICHMOND ROAD, WILLIAMSBURG, VA, 23185 - ..."
4,kabra2llc,KABRA-2 LLC,gregoryrdavis,GREGORY R DAVIS,"3051 HERITAGE LANDING RD, WILLIAMSBURG, VA, 23..."
...,...,...,...,...,...
899,goldenpondincspalonesamuels,"Golden Pond, Inc & Spalone, Samuel S",goldenpondincspalonesamuels,"Golden Pond, Inc & Spalone, Samuel S",
900,gorenfloseanwkimberlyb,"Gorenflo, Sean W & Kimberly B",gorenfloseanwkimberlyb,"Gorenflo, Sean W & Kimberly B",
901,goughwilliamhjr,"GOUGH, WILLIAM H JR",goughwilliamhjr,"GOUGH, WILLIAM H JR",
902,grahamfamilytrust,Graham Family Trust,graham_ft,Graham Family Trust,


In [ ]:
tOwner = pd.DataFrame(clean_merged[['owner_name', 'owner_name_display', 'ind_name', 'ind_name_display', 'company_address']].value_counts(dropna=False)).reset_index().drop(columns='count')
tOwner.columns = ['company_name', 'company_name_display', 'ind_name', 'ind_name_display', 'company_address']
tOwner.loc[tOwner['ind_name_display'].isna(), 'ind_name_display'] = tOwner.loc[tOwner['ind_name_display'].isna(), 'company_name_display']
tOwner.loc[tOwner['company_address'].isna(), 'company_address'] = ''
tOwner.to_csv('src/data/tOwner.csv', index=False)

In [ ]:
tProperty = clean_merged[['PID', 'owner_name', 'address', 'acres', 'total_value', 'usage',
                           'half_baths', 'full_baths', 'bedrooms', 'rental_units', 'image_path', 'address_display']]
tProperty.columns = ['pid', 'company_name', 'address', 'acres', 'total_value', 'usage',
       'half_baths', 'full_baths', 'bedrooms', 'rental_units', 'image_path', 'address_display']
# Drop duplicate addresses, not sure how they get in there, maybe listed mulitple times in original data
tProperty = tProperty.drop_duplicates()

# Two PIDs contain more than one address. For these two I just keep the row with the most rental_units
tProperty = tProperty.loc[tProperty.groupby('address')['rental_units'].idxmax()]
tProperty.to_csv('src/data/tProperty.csv', index=False)